# Pivot Tables With Pandas

## Introduction

In this section, we'll learn about the various ways we can index and structure our data sets to make it easier to process or understand.  We start by learning about the difference between the **_Wide_** format and the **_Long_** format, compare basic flattened index structures with multi-hierarchical index structures, and then create them ourselves using aggregation functions and pivot tables!

## Objectives

You will be able to:

* Understand and explain what a multi-level hierarchical index is
* Understand, explain the difference and use df.pivot and pd.pivot_table
* Switch between “long” and “wide” in a DataFrame using stack() and unstack()
* Transform “wide” to “long” DataFrames using `melt`


## Long and Wide Formats


These two different formats refer to how we structure our data in a DataFrame.  We'll start by comparing the two, and discussing the salient points of each. 

### Wide Format

This is the common setup you're probably used to seeing. In the wide format, each column of data represents a variable, and each row represents 1 observation.  If this were a medical dataset, each row would be the data corresponding to a different patient. 

In wide format, the index usually an integer, with 0 being the topmost row.  


### Long Format

In Long format, each index is a point in time for each observation.  See the following diagram for comparison of the two:

<br>
<br>
<img src="images/Image_200_wide_v_long.png">

Note that in this format, a given observation has values across multiple rows.  This is an especially useful format when dealing with time series data, or aggregated data, because we can make use of **_Multi-Hierarchical Indexing_**,

## Multi-Hierarchical Indexing

A common task when working with data is to create pivot tables, or group the data using aggregation functions.  In this respect, the Long format provides a very neat, organized way of structuring our data with multiple levels of indexes, allowing us to cleanly and easily represent different combinations of data.  Consider the following pivot table below (which you'll create yourself in the next lab):
<br>
<br>

<img src="images/pt1.png">

We can see by looking at the left two columns that this DataFrame has a Multi-Hierarchical Index with 2 levels, consisting of the "State" and Gender.  In this way, we have an easy way to aggregate and organize information to allow us to quickly answer questions with our data like "What was the total number of deaths by gender in Alabama?".

Let's take a look at one more example:


<img src="images/pt2.png">

In this DataFrame, the index has three hierarchical levels, with the outermost being "State" and the innermost being "Race".



## Pivot Tables

Pivot tables are a common toolset that you might have used before in spreadsheet software such as Microsoft Excel or Google Sheets.  

<img src="./images/excel_pt.png">
<br>
<center>_An Example Pivot Table created in Microsoft Excel_</center>

Pivot tables allow us to quickly examine our data by "pivoting" on different variables of interest.  Pandas allows us to easily create pivot tables with a built-in `.pivot()` method.

In order to create a pivot table, we need to just call the `dataframe.pivot()` method and specify the `index` column, the `columns` to put in our pivot table, and the `value` to put in each cell.  

For example, let's look at a pivot table you'll create in the next lab:

```python
some_dataframe.pivot(index='State', columns='Gender', values='Deaths_mean')
```

would return this pivot table:

<img src="./images/pt3.png">

Don't worry about the data this actually contains, as you don't yet have the context because you haven't been introduced to the dataset. Instead, just pay attention to the structure of the indexes.




## Stacking and Unstacking Data

One of the quickest ways to manipulate the format of a dataset in python is to use the `.stack()` and `unstack()` methods built into pandas DataFrames.  

Take a look at the following diagram and see if you can figure out what the `unstack()` is doing.

<img src='./images/Image_201_unstack.png'>

By telling the `.unstack()` call which index we want to unstack, we can move it from the index section over to the right as a variable column--`.stack()` would do the exact opposite, moving data to the left and making it a level of the index.  


In [ ]:
# Let's go back to our data we created in the previous chapter

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
columns = ['name', 'age', 'gender', 'job']
user1 = pd.DataFrame([['Giulia', 19, "F", "Actress"],
                      ['Josh', 26, "M", "Data Scientist"]],                  
                       columns=columns)
user1

,name,age,gender,job
0,Giulia,19,F,Actress
1,Josh,26,M,Data Scientist


In [4]:
user2 = pd.DataFrame([['Elisa', 22, "M", "student"],
                    ['Pablo', 58, "F", "Architect"]],
                     columns=columns)
user2

,name,age,gender,job
0,Elisa,22,M,student
1,Pablo,58,F,Architect


In [5]:
user3 = pd.DataFrame(dict(name=['Pietro', 'Saltini'],
                  age=[33, 44], gender=['M', 'F'],
                  job=['Farmer', 'Scientist']))
user3

,name,age,gender,job
0,Pietro,33,M,Farmer
1,Saltini,44,F,Scientist


In [6]:
user1.append(user2)
total_users = pd.concat([user1, user2 , user3])
total_users

,name,age,gender,job
0,Giulia,19,F,Actress
1,Josh,26,M,Data Scientist
0,Elisa,22,M,student
1,Pablo,58,F,Architect
0,Pietro,33,M,Farmer
1,Saltini,44,F,Scientist


# Reshaping by pivoting
+ “Unpivots” a DataFrame from wide format to long (stacked) format,

In [7]:
staked_df = pd.melt(total_users, id_vars="name", var_name="variable", value_name="value")
print(staked_df)

       name variable           value
0    Giulia      age              19
1      Josh      age              26
2     Elisa      age              22
3     Pablo      age              58
4    Pietro      age              33
5   Saltini      age              44
6    Giulia   gender               F
7      Josh   gender               M
8     Elisa   gender               M
9     Pablo   gender               F
10   Pietro   gender               M
11  Saltini   gender               F
12   Giulia      job         Actress
13     Josh      job  Data Scientist
14    Elisa      job         student
15    Pablo      job       Architect
16   Pietro      job          Farmer
17  Saltini      job       Scientist


# Using the `Pivot()`

+ “pivots” a DataFrame from long (stacked) format to wide format,

In [8]:
print(staked_df.pivot(index='name', columns='variable', values='value'))

variable age gender             job
name                               
Elisa     22      M         student
Giulia    19      F         Actress
Josh      26      M  Data Scientist
Pablo     58      F       Architect
Pietro    33      M          Farmer
Saltini   44      F       Scientist


## `.groupby()` and Aggregation Functions

Pandas DataFrames provide an easy way to group data using the `.groupby()` function.  To use this function, we just specify our indices in order (the columns we want to group the data by) and the function will return a new DataFrame containing the data grouped as we asked. 

For instance, if we wanted to group the Titanic Dataset by the port of embarkation, we would type:

```python
dataframe.groupby("Embarked")
```

This would give us a long format DataFrame with a flat index structure akin to the example in the top-most diagram.

Perhaps we want to group our data by both the port of embarkation and the ticket class (Labeled as _Pclass_ in the Titanic Dataset), we would type:

```python
dataframe.groupby(['Embarked', 'Pclass'])
```

This would return a DataFrame with a multi-hierarchical index, with `'Embarked'` being the outermost level.

Groupby statements are most commonly used with aggregation functions, which allow us to quickly calculate summary statistics such as the mean, median, min, max, mode, count, etc.  

Building on our last example, if we wanted to see the mean values for every grouping of `Embarked` and `Pclass` across different variables,  we would type:

```python
dataframe.groupby(['Embarked', 'Pclass']).mean()
```

Notice that we can make use of method chaining to quickly and concisely call the aggregation function at the same time as the groupby function.  


In [11]:
names_dict ={
    'Name':['Ken','Jeff','John','Mike','Andrew','Ann','Sylvia','Dorothy','Emily','Loyford'],
    'Age':[31,52,56,12,45,np.nan,78,85,46,135],
    'Phone':[52,79,80,75,43,125,74,44,85,45],
    'Uni':['One','Two','Three','One','Two','Three','One','Two','Three','One']
}

In [13]:
data = pd.DataFrame(names_dict)
data.head()

,Name,Age,Phone,Uni
0,Ken,31.0,52,One
1,Jeff,52.0,79,Two
2,John,56.0,80,Three
3,Mike,12.0,75,One
4,Andrew,45.0,43,Two


In [14]:
# Check for missing data
data.isna().sum()

Name     0
Age      1
Phone    0
Uni      0
dtype: int64

In [15]:
# Age has missing datat so let's fill it up
data['Age'].mean()

60.0

In [16]:
data["Age"].fillna(60, inplace=True)

# Let's use the groupby aggregate

In [17]:
data.groupby("Uni")

In [18]:
data.groupby(["Uni"])["Age"].mean()

Uni
One      64.000000
Three    54.000000
Two      60.666667
Name: Age, dtype: float64

In [19]:
# Or
data.groupby('Uni')['Age'].mean()

Uni
One      64.000000
Three    54.000000
Two      60.666667
Name: Age, dtype: float64

In [20]:
data.groupby(['Uni'])['Age'].mean().sort_values(ascending=False)

Uni
One      64.000000
Two      60.666667
Three    54.000000
Name: Age, dtype: float64

In [21]:
data.groupby(['Uni'])['Age'].mean().sort_values(ascending=False).reset_index()

,Uni,Age
0,One,64.000000
1,Two,60.666667
2,Three,54.000000


## Load the Data
The data for this activity is stored in a file called `'causes_of_death.tsv'` which is a somewhat morbid dataset from the center for disease control. Note that the file extension .tsv indicates that this data is formatted slightly differently then the standard .csv, the difference being that it has 'tab separated values' instead of 'comma separated values'. As such, pass in the optional parameter `delimiter='\t'` into the `pd.read_csv()` method.

In [10]:
df = pd.read_csv('./data/causes_of_death.tsv', delimiter = '\t')
df.head()

,Notes,State,State Code,Ten-Year Age Groups,Ten-Year Age Groups Code,Gender,Gender Code,Race,Race Code,Deaths,Population,Crude Rate
0,NaN,Alabama,1,< 1 year,1,Female,F,American Indian or Alaska Native,1002-5,14,3579,Unreliable
1,NaN,Alabama,1,< 1 year,1,Female,F,Asian or Pacific Islander,A-PI,24,7443,322.5
2,NaN,Alabama,1,< 1 year,1,Female,F,Black or African American,2054-5,2093,169339,1236.0
3,NaN,Alabama,1,< 1 year,1,Female,F,White,2106-3,2144,347921,616.2
4,NaN,Alabama,1,< 1 year,1,Male,M,Asian or Pacific Islander,A-PI,33,7366,448.0


Our data is currently in **_Wide_** format.  We can tidy this up by converting it to **_Long_** format by using groupby statements to aggregate our data into a much neater, more readable format. 

## Groupby Aggregations

Complete the following groupby statements.

### 1) Groupby State and Gender. Sum the values.

In [22]:
# Your code here
df.groupby(['State','Gender']).sum().head()

Notes  State Code  Deaths
State   Gender                           
Alabama Female    0.0          40  430133
        Male      0.0          41  430647
Alaska  Female    0.0          80   27199
        Male      0.0          84   36135
Arizona Female    0.0         180  396028

### 2) Groupby State and Gender and Race. Find the average values.

In [23]:
df.groupby(['State','Gender','Race']).mean().head()

Notes  State Code  \
State   Gender Race                                                  
Alabama Female American Indian or Alaska Native    NaN         1.0   
               Asian or Pacific Islander           NaN         1.0   
               Black or African American           NaN         1.0   
               White                               NaN         1.0   
        Male   American Indian or Alaska Native    NaN         1.0   

                                                       Deaths  
State   Gender Race                                            
Alabama Female American Indian or Alaska Native     70.875000  
               Asian or Pacific Islander            95.500000  
               Black or African American          9074.000000  
               White                             29890.636364  
        Male   American Indian or Alaska Native     86.375000

### 3) Groupby Gender and Race. Find the minimum values.

In [24]:
df.groupby(['Gender','Race']).min().head()

Notes    State  State Code  \
Gender Race                                                           
Female American Indian or Alaska Native    NaN  Alabama           1   
       Asian or Pacific Islander           NaN  Alabama           1   
       Black or African American           NaN  Alabama           1   
       White                               NaN  Alabama           1   
Male   American Indian or Alaska Native    NaN  Alabama           1   

                                        Ten-Year Age Groups  \
Gender Race                                                   
Female American Indian or Alaska Native           1-4 years   
       Asian or Pacific Islander                  1-4 years   
       Black or African American                  1-4 years   
       White                                      1-4 years   
Male   American Indian or Alaska Native           1-4 years   

                                        Ten-Year Age Groups Code Gender Code  \
Gender Race                                                                    
Female American Indian or Alaska Native                        1           F   
       Asian or Pacific Islander                               1           F   
       Black or African American                               1           F   
       White                                                   1           F   
Male   American Indian or Alaska Native                        1           M   

                                        Race Code  Deaths Population  \
Gender Race                                                            
Female American Indian or Alaska Native    1002-5      10     100285   
       Asian or Pacific Islander             A-PI      10     100371   
       Black or African American           2054-5      10     100170   
       White                               2106-3      12    1000704   
Male   American Indian or Alaska Native    1002-5      10       1001   

                                        Crude Rate  
Gender Race                                         
Female American Indian or Alaska Native       10.4  
       Asian or Pacific Islander              10.1  
       Black or African American             100.3  
       White                                  10.1  
Male   American Indian or Alaska Native    10000.0

# To be completed as separated exercise as I am tired.

In [ ]:

## Summary

We spent some time in this lesson learning about approaches to indexing and structuring our data sets. In the next lesson, we'll get some hands-on practice!